# devqubit: Getting Started

devqubit is a lightweight experiment tracker for quantum workflows. It helps you:

- **Track** parameters, metrics, tags, and artifacts for every run
- **Reproduce** results via fingerprints and stored program artifacts
- **Compare** runs (including statistical interpretation for shot noise)
- **Verify** candidates against a project baseline (great for CI)
- **Share** portable bundles (`.zip`) with everything needed to re-run or review

This notebook is intentionally small and "batteries-included": you can copy/paste the patterns into
your own experiments.

---

## What you'll build

1. Configure a workspace (store + registry)
2. Track a simple optimization run (params, metrics, artifacts)
3. Run a parameter sweep (grouped runs)
4. Compare runs and verify a candidate against a baseline
5. Package a run as a portable bundle and unpack it elsewhere


## 1. Storage configuration

devqubit stores everything in a **workspace**: run records plus a content-addressable object store.

You can configure the workspace in two ways (highest precedence first):

1. **In code** (explicit `store` / `registry` arguments to `track()`)
2. **Environment variables** (`DEVQUBIT_*`)

A typical local workspace looks like:

```
~/.devqubit/
├── objects/          # content-addressable object store
│   └── sha256/       # artifacts stored by digest
├── registry.db       # SQLite database for run metadata
└── baselines.json    # project baselines
```

For this demo we create a *temporary local folder* so you can run safely anywhere.


### Environment variables

The most common environment variables are:

- `DEVQUBIT_HOME` — workspace root directory (default: `~/.devqubit`)
- `DEVQUBIT_STORAGE_URL` — object store URL
- `DEVQUBIT_REGISTRY_URL` — registry URL
- `DEVQUBIT_CAPTURE_GIT` — capture git provenance (default: `true`)
- `DEVQUBIT_CAPTURE_PIP` — capture pip freeze (default: `false`)

In this notebook we use an **explicit local workspace** so everything stays self-contained.


In [ ]:
"""Setup: create an isolated local workspace for this demo."""

from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
import json
import shutil
import time
import numpy as np

# Public devqubit API
from devqubit import (
    track,
    diff,
    create_registry,
    create_store,
    verify_against_baseline,
    pack_run,
    unpack_bundle,
    Bundle,
)
from devqubit.compare import VerifyPolicy


@dataclass(frozen=True)
class DemoConfig:
    """Configuration for this notebook demo."""

    # Workspace
    workspace_dir: Path = Path(".devqubit_demo")

    # Reproducibility
    seed: int = 42

    # Mock optimization
    project: str = "optimization_study"
    iterations: int = 50

    # Sweep settings
    lr_sweep: tuple[float, ...] = (0.05, 0.10, 0.15)
    sweep_group_id: str = "lr_sweep_demo"
    sweep_group_name: str = "Learning Rate Sweep (Demo)"

    # Verification policy
    tvd_max: float = 0.10
    noise_factor: float = 3.0


CFG = DemoConfig()

# Reset demo workspace
if CFG.workspace_dir.exists():
    shutil.rmtree(CFG.workspace_dir)
CFG.workspace_dir.mkdir(parents=True, exist_ok=True)

# Create store and registry for this workspace
store = create_store(f"file://{CFG.workspace_dir}/objects")
registry = create_registry(f"file://{CFG.workspace_dir}")

np.random.seed(CFG.seed)

print(f"Workspace: {CFG.workspace_dir.resolve()}")

## 2. Basic tracking

The `track()` context manager is the main entry point. Inside the block you can log:

- **Parameters**: configuration / hyperparameters
- **Metrics**: scalars and time-series (via `step=...`)
- **Tags**: lightweight labels for filtering and grouping
- **Artifacts**: JSON, text, and binary files (e.g., circuits, counts, configs)

On exit, devqubit finalizes the run, computes fingerprints, and writes the record to the registry.


In [ ]:
def mock_optimization(learning_rate: float, iterations: int, seed: int) -> list[float]:
    """Simulate a simple optimization loop.

    Parameters
    ----------
    learning_rate
        Step size used to update the synthetic loss.
    iterations
        Number of iterations.
    seed
        Random seed for reproducibility.

    Returns
    -------
    list[float]
        Loss values over iterations.
    """
    rng = np.random.default_rng(seed)
    loss = 1.0
    history: list[float] = []
    for _ in range(iterations):
        loss *= 1 - learning_rate * rng.uniform(0.8, 1.0)
        history.append(float(loss))
    return history

In [ ]:
"""Run a single tracked experiment."""

with track(
    project=CFG.project,
    store=store,
    registry=registry,
    run_name="baseline",
) as run:
    # Parameters: things you'd want to reproduce exactly
    run.log_params(
        {
            "learning_rate": 0.10,
            "max_iterations": CFG.iterations,
            "optimizer": "sgd",
            "seed": CFG.seed,
        }
    )

    # Tags: convenient labels for filtering later
    run.set_tags({"demo": "getting_started", "kind": "baseline"})

    # Execute the experiment (here: synthetic optimization)
    t0 = time.time()
    history = mock_optimization(
        learning_rate=0.10,
        iterations=CFG.iterations,
        seed=CFG.seed,
    )
    duration_s = time.time() - t0

    # Time-series metric
    for step, loss in enumerate(history):
        run.log_metric("loss", loss, step=step)

    # Summary metrics
    run.log_metrics(
        {
            "final_loss": history[-1],
            "best_loss": float(min(history)),
            "duration_s": duration_s,
        }
    )

    # Artifacts (examples)
    run.log_json(
        name="run_config",
        obj={
            "learning_rate": 0.10,
            "iterations": CFG.iterations,
            "seed": CFG.seed,
        },
        role="config",
    )

    run.log_text(
        name="notes",
        text="Baseline run for getting-started demo.",
        role="documentation",
    )

    run.log_bytes(
        kind="loss_history.json",
        data=json.dumps(history).encode("utf-8"),
        media_type="application/json",
        role="results",
    )

    baseline_id = run.run_id

print(f"Baseline run_id: {baseline_id}")
print(f"Final loss:      {history[-1]:.6f}")

### Inspect the stored run

A run record can be loaded back from the registry at any time. Typical things to look at:

- `status` (FINISHED/FAILED)
- `params`, `metrics`, `tags`
- `artifacts` (with role/kind/media_type metadata)
- fingerprints (used for reproducibility checks)


In [ ]:
record = registry.load(baseline_id)

print(f"Run ID:   {record.run_id}")
print(f"Status:   {record.status}")

print("\nParams:")
for k, v in record.params.items():
    print(f"  {k}: {v}")

print("\nMetrics (last values):")
for k, v in record.metrics.items():
    print(f"  {k}: {v}")

print("\nTags:")
for k, v in record.tags.items():
    print(f"  {k}: {v}")

# Fingerprints
fingerprints = getattr(record, "fingerprints", {})
run_fp = fingerprints.get("run") or getattr(record, "run_fingerprint", None)
prog_fp = fingerprints.get("program") or getattr(record, "program_fingerprint", None)

print("\nFingerprints:")
print(f"  run:     {run_fp}")
print(f"  program: {prog_fp}")

## 3. Artifacts: list + retrieve

Artifacts are referenced by digest in the object store. The helper functions below make it easy to
list or find artifacts by role/kind, then download their bytes.


### Artifact roles (why they matter)

Artifacts are not just "files": their **role** tells devqubit how to interpret them.

Common roles:

- `program` — circuits/programs (important for reproducibility + comparisons)
- `results` — counts, expectation values, measurements
- `config` — compile/execute options, hyperparameters, settings
- `device_snapshot` — device calibration snapshots
- `documentation` — notes, reports, metadata for humans

In practice: **always** log your circuit/QASM/QPY under `role="program"` whenever possible.


In [ ]:
print("All artifacts:")
for art in record.artifacts:
    print(f"  [{art.role}] {art.kind} ({art.media_type}) -> {art.digest}")

# Find config artifact by role
config_art = next((a for a in record.artifacts if a.role == "config"), None)
if config_art:
    config_bytes = store.get_bytes(config_art.digest)
    print("\nConfig artifact (decoded):")
    print(config_bytes.decode("utf-8"))

## 4. Parameter sweep with grouped runs

When you sweep a parameter (shots, learning rate, depth, etc.), it's helpful to group runs:

- `group_id`: stable identifier you can query later
- `group_name`: human-readable label

This keeps your registry tidy and makes comparisons easier.


In [ ]:
sweep_run_ids: list[str] = []

for lr in CFG.lr_sweep:
    with track(
        project=CFG.project,
        store=store,
        registry=registry,
        group_id=CFG.sweep_group_id,
        group_name=CFG.sweep_group_name,
    ) as run:
        run.log_param("learning_rate", lr)
        run.log_param("max_iterations", CFG.iterations)
        run.set_tag("kind", "sweep")

        history = mock_optimization(
            learning_rate=lr,
            iterations=CFG.iterations,
            seed=CFG.seed,
        )
        run.log_metric("final_loss", history[-1])
        run.log_metric("best_loss", float(min(history)))

        sweep_run_ids.append(run.run_id)

print("Sweep runs:")
for rid in sweep_run_ids:
    print(" ", rid)

### List groups and their runs


In [ ]:
groups = registry.list_groups()

print("Groups:")
for g in groups:
    print(f"  {g['group_id']} - {g['group_name']} ({g['project']})")

print("\nRuns in sweep group:")
runs_in_group = registry.list_runs_in_group(CFG.sweep_group_id)
for r in runs_in_group:
    print(r)

## 5. Parent → child run lineage (multi-stage experiments)

For workflows like:
- coarse search → fine search
- baseline → candidate tuning
- training → evaluation

...you can link runs via `parent_run_id`.


In [ ]:
with track(
    project=CFG.project,
    store=store,
    registry=registry,
    run_name="coarse",
) as parent:
    parent.log_param("stage", "coarse")
    parent_id = parent.run_id

with track(
    project=CFG.project,
    store=store,
    registry=registry,
    run_name="fine",
    parent_run_id=parent_id,
) as child:
    child.log_param("stage", "fine")
    child.log_param("parent_run_id", parent_id)

print(f"Parent: {parent_id}")
print(f"Child:  {child.run_id}")

## 6. Search (filter runs by params/metrics/tags)

The registry supports simple query expressions to find runs of interest.


In [ ]:
# Example 1: parameter-based search
results = registry.search_runs("params.learning_rate = 0.1")
print(f"params.learning_rate = 0.1 -> {len(results)} runs")

# Example 2: metric-based search (final_loss exists on sweep runs)
results = registry.search_runs("metric.final_loss > 0")
print(f"metric.final_loss > 0 -> {len(results)} runs")

# Example 3: sort by a metric
results = registry.search_runs(
    "metric.final_loss > 0",
    sort_by="metric.final_loss",
    limit=5,
)
print("\nTop 5 by final_loss:")
for r in results:
    print(
        f"  {r.run_id} | lr={r.params.get('learning_rate')} | final_loss={r.metrics.get('final_loss')}"
    )

## 7. Baselines, comparison, and verification

devqubit supports two related workflows:

- **Compare**: diff two runs and interpret result differences (e.g., shot-noise-aware TVD).
- **Verify**: check a candidate run against the project's baseline using a policy (useful in CI).

> In quantum workloads, distributions often vary run-to-run due to sampling noise.
> devqubit uses statistics (e.g., expected noise) to help interpret TVD and avoid false alarms.


### A note on TVD (Total Variation Distance)

When you compare two measured distributions (e.g., bitstring counts), a common distance is **TVD**:

\[
\mathrm{TVD}(p, q) = \frac{1}{2} \sum_x |p(x) - q(x)|
\]

- **0.0** means identical distributions
- small values are often consistent with **shot noise**
- larger values may indicate a real change (code/config/device drift)

devqubit uses this (plus an expected-noise model) to avoid false alarms in CI.


In [ ]:
"""Set the baseline for this project."""

registry.set_baseline(CFG.project, baseline_id)
baseline_info = registry.get_baseline(CFG.project)
print("Baseline set:")
print(baseline_info)

In [ ]:
"""Create a candidate run and compare it to baseline."""

with track(
    project=CFG.project,
    store=store,
    registry=registry,
    run_name="candidate",
) as run:
    run.log_params(
        {
            "learning_rate": 0.12,
            "max_iterations": CFG.iterations,
            "seed": CFG.seed,
        }
    )
    run.set_tag("kind", "candidate")

    history = mock_optimization(
        learning_rate=0.12,
        iterations=CFG.iterations,
        seed=CFG.seed,
    )
    run.log_metric("final_loss", history[-1])
    candidate_id = run.run_id

print(f"Candidate run_id: {candidate_id}")

# Compare baseline vs candidate
comparison = diff(
    baseline_id,
    candidate_id,
    registry=registry,
    store=store,
)

print(comparison)

In [ ]:
"""Verify the candidate against the project's baseline."""

policy = VerifyPolicy(
    params_must_match=False,  # candidate may differ in params
    program_must_match=False,  # this toy example has no program artifact
    tvd_max=CFG.tvd_max,
    noise_factor=CFG.noise_factor,
)

verify_result = verify_against_baseline(
    candidate=registry.load(candidate_id),
    project=CFG.project,
    store=store,
    registry=registry,
    policy=policy,
)

print(verify_result)

## 8. Bundles: share runs portably

Bundles are zip files containing:

- the run record (params/metrics/tags/artifacts metadata)
- all referenced objects (by digest)

This makes it easy to share runs across machines or attach them to CI artifacts.


In [ ]:
bundle_path = CFG.workspace_dir / f"{baseline_id[:8]}.devqubit.zip"

pack_result = pack_run(
    run_id=baseline_id,
    output_path=bundle_path,
    store=store,
    registry=registry,
)

print(f"Bundle created: {bundle_path.name}")
print(f"  Size:      {bundle_path.stat().st_size / 1024:.1f} KB")
print(f"  Artifacts: {pack_result.artifact_count}")
print(f"  Objects:   {pack_result.object_count}")

In [ ]:
"""Inspect bundle contents without extracting."""

with Bundle(bundle_path) as b:
    print("Bundle manifest:")
    print(json.dumps(b.manifest, indent=2)[:800] + "\n...")

    print("\nFirst few objects:")
    for obj in b.list_objects()[:5]:
        print(" ", obj)

In [ ]:
"""Unpack the bundle into a new workspace and sanity-check it."""

import_dir = CFG.workspace_dir / "imported"
import_dir.mkdir(exist_ok=True)

import_store = create_store(f"file://{import_dir}/objects")
import_registry = create_registry(f"file://{import_dir}")

unpack_result = unpack_bundle(
    bundle_path=bundle_path,
    dest_store=import_store,
    dest_registry=import_registry,
)

print(f"Unpacked run: {unpack_result.run_id}")

# Quick integrity checks
imported = import_registry.load(unpack_result.run_id)
original = registry.load(baseline_id)

print(f"Params match:  {imported.params == original.params}")
print(f"Tags match:    {imported.tags == original.tags}")

## 9. Clean up (optional)

Delete the local demo workspace created by this notebook.


In [ ]:
# Comment this out if you want to keep the demo workspace around.
shutil.rmtree(CFG.workspace_dir)
print("Done.")